# A Transformer-based recommendation system


In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import pandas as pd
from src.dataset import RatingDataset
from src import utils
from torch.utils.data import DataLoader
from torch import nn
import math
import torch

### (1) Load Dataset

In [3]:
df_train = pd.read_parquet("./artifacts/train_data.parquet")
df_test = pd.read_parquet("./artifacts/test_data.parquet")

In [4]:
train_dataset = RatingDataset(data=df_train) 

In [5]:
loader = DataLoader(train_dataset,batch_size=12,shuffle=True)

In [6]:
for inputs in loader:
    break


## (2) Model Config

In [7]:
# inputs

In [8]:
age_group_id_map_dict = utils.open_object("./artifacts/age_group_id_map_dict.pkl")

movie_id_map_dict = utils.open_object("./artifacts/movie_id_map_dict.pkl")

occupation_id_map_dict = utils.open_object("./artifacts/occupation_id_map_dict.pkl")

sex_id_map_dict = utils.open_object("./artifacts/sex_id_map_dict.pkl")

user_id_map_dict = utils.open_object("./artifacts/user_id_map_dict.pkl")
# genres_map_dict = utils.open_object("./artifacts/genres_map_dict.pkl")

In [9]:
rating_min_max_scaler = utils.open_object("./artifacts/rating_min_max_scaler.pkl")

In [10]:
num_user = len(user_id_map_dict)
num_movie = len(movie_id_map_dict)
num_occupation = len(occupation_id_map_dict)
num_age_group = len(age_group_id_map_dict)
# num_genre = len(genres_map_dict)

In [11]:
int(math.sqrt(num_user))

77

In [12]:
int(math.sqrt(num_occupation))

4

In [13]:
embed_configs = {}
EMED_DIM=64
embed_configs['user']={"embed_dim":EMED_DIM,"num_embed":num_user}
embed_configs['movie']={"embed_dim":EMED_DIM,"num_embed":num_movie}
embed_configs['occupation']={"embed_dim":int(math.sqrt(num_occupation)),"num_embed":num_occupation}
embed_configs['age_group']={"embed_dim":int(math.sqrt(num_age_group)),"num_embed":num_age_group}

In [14]:
sequence_length=4
embed_configs['position'] = {'embed_dim': EMED_DIM, 'num_embed': sequence_length-1}

In [15]:
embed_configs

{'user': {'embed_dim': 64, 'num_embed': 6041},
 'movie': {'embed_dim': 64, 'num_embed': 3884},
 'occupation': {'embed_dim': 4, 'num_embed': 22},
 'age_group': {'embed_dim': 2, 'num_embed': 8},
 'position': {'embed_dim': 64, 'num_embed': 3}}

In [16]:
config_dict={}
config_dict['embed_configs'] = embed_configs
config_dict['sequence_length'] = sequence_length

In [17]:
class Config:
    def __init__(self, dictionary):
        for key, value in dictionary.items():
            setattr(self, key, value)

In [18]:
config = Config(dictionary=config_dict)

In [19]:
config.embed_configs

{'user': {'embed_dim': 64, 'num_embed': 6041},
 'movie': {'embed_dim': 64, 'num_embed': 3884},
 'occupation': {'embed_dim': 4, 'num_embed': 22},
 'age_group': {'embed_dim': 2, 'num_embed': 8},
 'position': {'embed_dim': 64, 'num_embed': 3}}

## Modeling

### (1) Create Embedding Layer

In [20]:
class self:
    pass

In [21]:
self.config = config

In [22]:
embed_configs = self.config.embed_configs

embedding_layers = []
for name,embed_config in embed_configs.items():
    embed_dim = embed_config['embed_dim']
    num_embed = embed_config['num_embed']
    embeding_layer = nn.Embedding(num_embeddings=num_embed, embedding_dim=embed_dim)
    nn.init.xavier_uniform_(embeding_layer.weight)
    embedding_layers.append([name,embeding_layer])

self.embedding_layers = nn.ModuleDict(embedding_layers)

In [23]:
self.embedding_layers

ModuleDict(
  (user): Embedding(6041, 64)
  (movie): Embedding(3884, 64)
  (occupation): Embedding(22, 4)
  (age_group): Embedding(8, 2)
  (position): Embedding(3, 64)
)

In [24]:
inputs.keys()

dict_keys(['user_id_index', 'movie_sequence', 'rating_sequence', 'sex', 'occupation_index', 'age_group_index', 'target_rating', 'target_movie'])

### (2) Embed Movie

In [25]:
inputs['target_movie']

tensor([2702, 3851, 1996, 1223,  902, 3546, 1223, 1905,   36, 3292, 1263, 2938])

In [26]:
target_movie_embedding = self.embedding_layers['movie'](inputs['target_movie'])

In [27]:
target_movie_embedding.shape

torch.Size([12, 64])

In [28]:
movie_sequence_embedding  = self.embedding_layers['movie'](inputs['movie_sequence'])

### (3) Encode Movie Embedding

In [29]:

class MovieEmbedEncoder(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.linear = nn.Linear(input_size, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.linear(x)
        x = self.relu(x)
        return x

In [30]:
movie_embedding_size = self.config.embed_configs['movie']['embed_dim']

In [31]:
self.movie_embed_encoder = MovieEmbedEncoder(input_size=movie_embedding_size,output_size=movie_embedding_size)

In [32]:
target_movie_encoded = self.movie_embed_encoder(target_movie_embedding)

In [34]:
movie_sequence_encoded = self.movie_embed_encoder(movie_sequence_embedding)

### (4) Position Embedding

In [35]:
positions = torch.arange(self.config.sequence_length-1)

In [36]:
position_embedding = self.embedding_layers['position'](positions)

### (5) Combine Position and Sequence Rating 

In [37]:
movie_sequence_encoded  = movie_sequence_encoded + position_embedding

In [38]:
rating_sequence = inputs['rating_sequence']

In [39]:
# element wise product with rating
encoded_sequence_movies_with_poistion_and_rating = torch.mul(movie_sequence_encoded, rating_sequence.unsqueeze(-1))

In [40]:
encoded_sequence_movies_with_poistion_and_rating.shape

torch.Size([12, 3, 64])

### (6) Transformer for sequence features

In [41]:
encoder_layer = nn.TransformerEncoderLayer(d_model=movie_embedding_size, nhead=8)
self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=3)

In [43]:
transformed_sequence= self.transformer_encoder(encoded_sequence_movies_with_poistion_and_rating)

In [44]:
transformed_sequence  = transformed_sequence.permute(0,2,1)

In [46]:
avg_pool_layer = torch.nn.AdaptiveAvgPool1d(1)

In [53]:
sequence_features = avg_pool_layer(transformed_sequence).squeeze(-1)

### (7) Combine with other Features

In [56]:
self.embedding_layers

ModuleDict(
  (user): Embedding(6041, 64)
  (movie): Embedding(3884, 64)
  (occupation): Embedding(22, 4)
  (age_group): Embedding(8, 2)
  (position): Embedding(3, 64)
)

In [55]:
user_embedding = self.embedding_layers['user'](inputs['user_id_index'])

In [58]:
occupation_embedding = self.embedding_layers['occupation'](inputs['occupation_index'])

In [73]:
user_features = torch.concat([user_embedding,occupation_embedding,inputs['sex'].view(-1,1)],dim=1)

In [74]:
user_features.shape

torch.Size([12, 69])

In [80]:
import torch.nn as nn
import torch.nn.functional as F

class FeedForward(nn.Module):
    def __init__(self, d_model, dff, dropout_rate=0.1):
        super(FeedForward, self).__init__()
        self.ff1 = nn.Linear(d_model, dff)
        self.ff2 = nn.Linear(dff, d_model)
        self.dropout = nn.Dropout(dropout_rate)
        self.norm = nn.LayerNorm(d_model)

    def forward(self, x):
        ff = F.relu(self.ff1(x))
        ff = self.ff2(ff)
        ff = self.dropout(ff)
        x = self.norm(x + ff)
        return x
